In [21]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten

In [29]:
IMAGE_SIZE = 224
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 25

In [7]:
# Load MobileNet model
conv_base = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
)

9406464/9406464 [==============================] - 7s 1us/step


In [13]:
conv_base.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_2[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[

In [46]:
#for layer in conv_base.layers:
 #   layer.trainable = False

for i, layer in enumerate (conv_base.layers):
    print(i, layer.name, layer.trainable)

0 input_2 False
1 Conv1 False
2 bn_Conv1 False
3 Conv1_relu False
4 expanded_conv_depthwise False
5 expanded_conv_depthwise_BN False
6 expanded_conv_depthwise_relu False
7 expanded_conv_project False
8 expanded_conv_project_BN False
9 block_1_expand False
10 block_1_expand_BN False
11 block_1_expand_relu False
12 block_1_pad False
13 block_1_depthwise False
14 block_1_depthwise_BN False
15 block_1_depthwise_relu False
16 block_1_project False
17 block_1_project_BN False
18 block_2_expand False
19 block_2_expand_BN False
20 block_2_expand_relu False
21 block_2_depthwise False
22 block_2_depthwise_BN False
23 block_2_depthwise_relu False
24 block_2_project False
25 block_2_project_BN False
26 block_2_add False
27 block_3_expand False
28 block_3_expand_BN False
29 block_3_expand_relu False
30 block_3_pad False
31 block_3_depthwise False
32 block_3_depthwise_BN False
33 block_3_depthwise_relu False
34 block_3_project False
35 block_3_project_BN False
36 block_4_expand False
37 block_4_expa

In [34]:
# Freeze layers in MobileNet except the last few layers
#for layer in conv_base.layers:
 #   layer.trainable = False

# Freeze most layers
#for layer in conv_base.layers[:-10]:
 #   layer.trainable = False

# Fine-tune the last 10 layers
#for layer in conv_base.layers[-10:]:
 #   layer.trainable = True

In [47]:
conv_base.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_2[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[

In [48]:
# Define your model
model = models.Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [49]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 flatten_2 (Flatten)         (None, 62720)             0         
                                                                 
 dense_9 (Dense)             (None, 64)                4014144   
                                                                 
 dense_10 (Dense)            (None, 128)               8320      
                                                                 
 dense_11 (Dense)            (None, 2)                 258       
                                                                 
Total params: 6280706 (23.96 MB)
Trainable params: 4755202 (18.14 MB)
Non-trainable params: 1525504 (5.82 MB)
__________

In [50]:
# Data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load training data
train_generator = datagen.flow_from_directory(
    r"C:\Users\DELL\Desktop\TINT\Deep Learning\Pneumonia Detection\Chest_Xray\train",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

# Load validation data
validation_generator = datagen.flow_from_directory(
    r"C:\Users\DELL\Desktop\TINT\Deep Learning\Pneumonia Detection\Chest_Xray\valid",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 12229 images belonging to 2 classes.
Found 1747 images belonging to 2 classes.


In [51]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [52]:
# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator
)

Epoch 1/25
383/383 [==============================] - 370s 951ms/step - loss: 0.1363 - accuracy: 0.9515 - val_loss: 0.2991 - val_accuracy: 0.9508
Epoch 2/25
383/383 [==============================] - 351s 917ms/step - loss: 0.1137 - accuracy: 0.9598 - val_loss: 0.3970 - val_accuracy: 0.9399
Epoch 3/25
383/383 [==============================] - 333s 871ms/step - loss: 0.1074 - accuracy: 0.9594 - val_loss: 0.2931 - val_accuracy: 0.9473
Epoch 4/25
383/383 [==============================] - 349s 910ms/step - loss: 0.1040 - accuracy: 0.9631 - val_loss: 0.1575 - val_accuracy: 0.9548
Epoch 5/25
383/383 [==============================] - 349s 913ms/step - loss: 0.0993 - accuracy: 0.9625 - val_loss: 0.1402 - val_accuracy: 0.9571
Epoch 6/25
383/383 [==============================] - 360s 939ms/step - loss: 0.1035 - accuracy: 0.9603 - val_loss: 0.2345 - val_accuracy: 0.9399
Epoch 7/25
383/383 [==============================] - 296s 773ms/step - loss: 0.1000 - accuracy: 0.9625 - val_loss: 0.1543 -

In [1]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='accuracy', color='blue')
plt.plot(history.history['val_accuracy'], label='val_accuracy', color='red')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
# plt.legend()
plt.show()

NameError: name 'history' is not defined

In [54]:
model_version="mobilenet2"
model.save(f"C:\\Users\\DELL\\Desktop\\TINT\\Deep Learning\\Pneumonia Detection\\Model\\{model_version}")

INFO:tensorflow:Assets written to: C:\Users\DELL\Desktop\TINT\Deep Learning\Pneumonia Detection\Model\mobilenet2\assets


INFO:tensorflow:Assets written to: C:\Users\DELL\Desktop\TINT\Deep Learning\Pneumonia Detection\Model\mobilenet2\assets


In [55]:
# Precision and Recall metrics
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

In [57]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Calculate precision and recall separately
predictions = model.predict(validation_generator)
true_labels = validation_generator.classes

# Calculate precision and recall using sklearn.metrics
from sklearn.metrics import precision_score, recall_score

precision = precision_score(true_labels, predictions.argmax(axis=1))
recall = recall_score(true_labels, predictions.argmax(axis=1))

print(f'Precision: {precision}, Recall: {recall}')

55/55 [==============================] - 31s 565ms/step - loss: 0.1351 - accuracy: 0.9571
Loss: 0.1350805014371872, Accuracy: 0.9570692777633667
55/55 [==============================] - 32s 569ms/step
Precision: 0.7352264006139677, Recall: 0.7531446540880503


In [59]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
import numpy as np
# Get true labels and predicted labels for validation data
val_data, val_labels = validation_generator.next()
val_pred_probs = model.predict(val_data)
val_pred_labels = np.argmax(val_pred_probs, axis=1)  # Convert predicted probabilities to labels

# Calculate accuracy
accuracy = accuracy_score(np.argmax(val_labels, axis=1), val_pred_labels)

# Calculate precision
precision = precision_score(np.argmax(val_labels, axis=1), val_pred_labels)

# Calculate recall
recall = recall_score(np.argmax(val_labels, axis=1), val_pred_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

# You can also print a classification report which includes precision, recall, and F1-score for each class
print(classification_report(np.argmax(val_labels, axis=1), val_pred_labels))

1/1 [==============================] - 0s 398ms/step
Accuracy: 0.96875
Precision: 0.9642857142857143
Recall: 1.0
              precision    recall  f1-score   support

           0       1.00      0.80      0.89         5
           1       0.96      1.00      0.98        27

    accuracy                           0.97        32
   macro avg       0.98      0.90      0.94        32
weighted avg       0.97      0.97      0.97        32

